# Busines Problem

Understanding the needs of the customers is a very important for those who want to start a business. Although the needs are important, the most important aspect to have in mind is to consider the socioeconomic backgrounds of the possible target group. For example, although people might need accomodation, they might not be able to buy one as their economic resources might not be enough. Thus, what is important it to direct what kind of socioeconomic status they have. 

The business idea of this project is to understand the buying behaviours of different neighboorhoods in Munich and based on that suggest stakeholders to start business that can address to those people living in such neighborhoods budgets. As know in Germany, people who have middle and lower socioeconomic backgrounds generally shop from such supermarkets as Aldi or Lidl. Besides, it is mostly known that those people eat at McDonalds. Therefore, it will be more appropriate to build shops adressing to poor or middle class neighborhoods and Rewe or Tengelman to high class neighborhoods. 


# How to Use the Location Data

Munich location data and its neighboohds data will be reached using Foursquare API and the neighborhoods will be clustered together based on their socioeconomic status (as there are certain neighborhoods where mostly rich and others where mostly poor people live). Based on the clustered neighborhoods, the number of Aldi, Lidl, Tengelman and Rewe in each cluster will be calculated. Further, their spatial proximities will also be added in when suggestion a stakeholder to build a certain shop at a certain neighborhood. 
